# Descritores de Imagens via Histograma de Cor

Exemplos baseados em https://github.com/xn2333/OpenCV/blob/master/Seminar_Image_Processing_in_Python.ipynb*







# Instalando Bibliotecas

In [1]:
import numpy as np
import pandas as pd
import cv2 as cv 
from google.colab.patches import cv2_imshow # for image display
from skimage import io
from PIL import Image 
import matplotlib.pylab as plt

# Vamos construir nosso dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil, sys

# move used images
dataset = pd.read_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_dataset.csv')
print(dataset)
for index, row in dataset.iterrows():
  if row['post']:
    shutil.move("/content/drive/MyDrive/hourlyloonacatcher/" + row['group'] + "/" + row['filename'], "/content/drive/MyDrive/hourlyloonacatcher/" + row['group'] + "/posted/")
    dataset = dataset.drop([index])

dataset = dataset.reset_index().drop(columns='index')
print(dataset)
dataset.to_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_dataset.csv')

       Unnamed: 0   group         filename   post
0               0   loona   jinsoul130.jpg  False
1               1   loona   jinsoul135.jpg  False
2               2   loona   jinsoul134.jpg   True
3               3   loona   jinsoul132.jpg   True
4               4   loona   jinsoul136.jpg   True
...           ...     ...              ...    ...
12325       12325  deukae   siyeon1017.jpg  False
12326       12326  deukae      dami840.jpg  False
12327       12327  deukae  yoohyeon530.jpg  False
12328       12328  deukae    siyeon888.jpg  False
12329       12329  deukae      dami788.jpg  False

[12330 rows x 4 columns]
       Unnamed: 0   group         filename   post
0               0   loona   jinsoul130.jpg  False
1               1   loona   jinsoul135.jpg  False
2               7   loona   jinsoul137.jpg  False
3               8   loona   jinsoul140.jpg  False
4              10   loona   jinsoul142.jpg  False
...           ...     ...              ...    ...
10021       12325  deuka

In [3]:
# percentagens taken by each girl

loona = [
      'heejin',
      'hyunjin',
      'haseul',
      'yeojin',
      'vivi',
      'kimlip',
      'jinsoul',
      'choerry',
      'yves',
      'chuu',
      'gowon',
      'oliviahye'
  ]

deukae = [
    "jiu",
    "sua",
    "siyeon",
    "handong",
    "yoohyeon",
    "dami",
    "gahyeon"
]

images = pd.read_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/images_to_post.csv')

for l in loona:
    print('{0}: {1}'.format(l,(100 * len(images[images['loona'].str.contains(l)].index) / len(images.index))))

print()

for d in deukae:
    print('{0}: {1}'.format(d,(100 * len(images[images['deukae'].str.contains(d)].index) / len(images.index))))   


heejin: 7.958477508650519
hyunjin: 9.342560553633218
haseul: 8.650519031141869
yeojin: 9.342560553633218
vivi: 7.958477508650519
kimlip: 6.920415224913495
jinsoul: 9.688581314878892
choerry: 8.996539792387543
yves: 8.304498269896193
chuu: 6.57439446366782
gowon: 8.650519031141869
oliviahye: 7.612456747404845

jiu: 12.802768166089965
sua: 14.53287197231834
siyeon: 14.53287197231834
handong: 13.494809688581315
yoohyeon: 15.22491349480969
dami: 14.186851211072664
gahyeon: 15.22491349480969


In [ ]:
loona = [
      'heejin',
      'hyunjin',
      'haseul',
      'yeojin',
      'vivi',
      'kimlip',
      'jinsoul',
      'choerry',
      'yves',
      'chuu',
      'gowon',
      'oliviahye'
  ]

deukae = [
    "jiu",
    "sua",
    "siyeon",
    "handong",
    "yoohyeon",
    "dami",
    "gahyeon"
]

pairs = {}
for l in loona:
  for d in deukae:
    pairs[l + '_' + d] = 0

post_log = pd.read_csv('/content/drive/MyDrive/hourlyloonacatcher/post_log.csv')

for index, row in post_log.iterrows():
  pairs[row['loona'] + '_' + row['deukae']] += 1

pairs_df = {'girls': [], 'values': []}
for key, value in pairs.items():
  pairs_df['girls'].append(key)
  pairs_df['values'].append(100 * (value / len(post_log.index)))

pairs_df = pd.DataFrame(pairs_df)
pairs_df.to_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/log_stats.csv')

In [ ]:
def load_zero():
  import os

  path = '/content/drive/MyDrive/hourlyloonacatcher/'

  dataset = {"group": [], "filename": [], "image": [], "post": []}
  for filename in os.listdir(path + 'loona/'):
      img = cv.imread(path + 'loona/' + filename)
      if img is not None:
          dataset["group"].append('loona')
          dataset["filename"].append(filename)
          dataset["image"].append(img)
          dataset["post"].append(False)

  for filename in os.listdir(path + 'deukae/'):
      img = cv.imread(path + 'deukae/' + filename)
      if img is not None:
          dataset["group"].append('deukae')
          dataset["filename"].append(filename)
          dataset["image"].append(img)
          dataset["post"].append(False)

  return dataset

def load_flying():
  import os

  path = '/content/drive/MyDrive/hourlyloonacatcher/'

  dataset = pd.read_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_dataset.csv').drop(columns='Unnamed: 0').reset_index().drop(columns='index').to_dict(orient='list')
  dataset["image"] = []
  for filename in os.listdir(path + 'loona/'):
      img = cv.imread(path + 'loona/' + filename)
      if img is not None and filename not in dataset["filename"]:
        dataset["group"].append('loona')
        dataset["filename"].append(filename)
        dataset["image"].append(img)
        dataset["post"].append(False)
      elif img is not None:
        dataset["image"].append(img)

  for filename in os.listdir(path + 'deukae/'):
      img = cv.imread(path + 'deukae/' + filename)
      if img is not None and filename not in dataset["filename"]:
        dataset["group"].append('deukae')
        dataset["filename"].append(filename)
        dataset["image"].append(img)
        dataset["post"].append(False)
      elif img is not None:
        dataset["image"].append(img)

  return dataset

# Extraindo características do dataset usando Histograma de Cor

In [ ]:
def prepareX(dataset):
  color = ('b','g','r')

  dataset_hist_r = []
  dataset_hist_g = []
  dataset_hist_b = []

  counter = 0
  for image in dataset["image"]:
    hists = {}
    for i,col in enumerate(color):
      histr = cv.calcHist([image],[i],None,[256],[0,256])
      if col == 'r': dataset_hist_r.append(histr)
      if col == 'g': dataset_hist_g.append(histr)
      if col == 'b': dataset_hist_b.append(histr)


  X_r = np.array(dataset_hist_r)
  length = np.sqrt((X_r**2).sum(axis=1))[:,None]
  X_r = X_r / length

  X_g = np.array(dataset_hist_g)
  length = np.sqrt((X_g**2).sum(axis=1))[:,None]
  X_g = X_g / length


  X_b = np.array(dataset_hist_b)
  length = np.sqrt((X_b**2).sum(axis=1))[:,None]
  X_b = X_b / length

  X = np.concatenate((X_r,X_g,X_g),axis=1)
  X.shape

  X = X.reshape(X.shape[0],X.shape[1])
  X.shape

  return X

# Agrupamento de Imagens

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

def do_kmeans(X, n_clusters=100):
  kmeans = KMeans(n_clusters=n_clusters).fit(X)
  return kmeans

## Escolher a imagem

In [ ]:
def get_probs(loona_prob = [100/12] * 12, deukae_prob = [100/7] * 7):
  loona = [
      'heejin',
      'hyunjin',
      'haseul',
      'yeojin',
      'vivi',
      'kimlip',
      'jinsoul',
      'choerry',
      'yves',
      'chuu',
      'gowon',
      'oliviahye'
  ]

  deukae = [
      "jiu",
      "sua",
      "siyeon",
      "handong",
      "yoohyeon",
      "dami",
      "gahyeon"
  ]

  return loona, loona_prob, deukae, deukae_prob

In [ ]:
def update_probs(loona_prob, deukae_prob, loona, deukae, loona_choice, deukae_choice):
  for i in range(len(loona)):
    if loona[i] not in loona_choice:
      loona_prob[i] += 100/12
    else:
      loona_prob[i] = 100/12

  for i in range(len(deukae)):
    if deukae[i] not in deukae_choice:
      deukae_prob[i] += 100/7
    else:
      deukae_prob[i] = 100/7

  return loona_prob, deukae_prob

In [ ]:
import random 
from sklearn.metrics import silhouette_samples

n_clusters = 50

def first_run():  

  dataset = load_zero()
  X = prepareX(dataset)
  kmeans = do_kmeans(X, n_clusters) 
  cluster_labels = kmeans.labels_

  sample_silhouette_values = silhouette_samples(X, cluster_labels)

  means_lst = []
  for label in range(n_clusters):
      means_lst.append(sample_silhouette_values[cluster_labels == label].mean())

  clusterList = [x for _,x in sorted(zip(means_lst,range(n_clusters)))]

  imageLoona = None
  imageDeukae = None

  images_to_post = {'loona': [], 'deukae': []}

  # first time

  loona, loona_prob, deukae, deukae_prob = get_probs()
  loona_choice = random.choices(loona, weights=loona_prob, k=1) 
  deukae_choice = random.choices(deukae, weights=deukae_prob, k=1) 
  loona_prob, deukae_prob = update_probs(loona_prob, deukae_prob, loona, deukae, loona_choice, deukae_choice)

  for i in range(192):
    print(loona_choice[0], deukae_choice[0]) 
    imageLoona, imageDeukae = -1, -1
    #get image pair for post
    for c in clusterList:
      for image_id, cluster in enumerate(cluster_labels):
          if not dataset["post"][image_id]:
            if cluster == c:
              if loona_choice[0] in dataset["filename"][image_id]:
                print(image_id)
                imageLoona = image_id
                break

      for image_id, cluster in enumerate(cluster_labels):
          if not dataset["post"][image_id]:
            if cluster == c:
              if deukae_choice[0] in dataset["filename"][image_id]:
                print(image_id)
                imageDeukae = image_id
                break

      if (imageLoona != -1) and (imageDeukae != -1):
        dataset["post"][imageLoona] = True
        dataset["post"][imageDeukae] = True
        # new pair
        loona_choice = random.choices(loona, weights=loona_prob, k=1) 
        deukae_choice = random.choices(deukae, weights=deukae_prob, k=1) 
        loona_prob, deukae_prob = update_probs(loona_prob, deukae_prob, loona, deukae, loona_choice, deukae_choice)
        # get out and choose right image
        break
    
    if (imageLoona != -1) and (imageDeukae != -1):
      images_to_post['loona'].append(dataset["filename"][imageLoona])
      images_to_post['deukae'].append(dataset["filename"][imageDeukae])
    else:
      print("No more image combinations for the pair")

    #cv2_imshow(dataset["image"][imageLoona])
    #cv2_imshow(dataset["image"][imageDeukae])

  # generate dfs for saving

  del dataset['image']
  df_dataset = pd.DataFrame(dataset)

  df_images_to_post = pd.DataFrame(images_to_post)

  df_loona = {'name': [], 'prob': []}
  df_loona['name'] = loona
  df_loona['prob'] = loona_prob
  df_loona = pd.DataFrame(df_loona)

  df_deukae = {'name': [], 'prob': []}
  df_deukae['name'] = deukae
  df_deukae['prob'] = deukae_prob
  df_deukae = pd.DataFrame(df_deukae)

  df_dataset.to_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_dataset.csv')
  df_images_to_post.to_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/images_to_post.csv')
  df_loona.to_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_loona.csv')
  df_deukae.to_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_deukae.csv')

def flying_run():  

  dataset = load_flying()
  X = prepareX(dataset)
  kmeans = do_kmeans(X, n_clusters) 
  cluster_labels = kmeans.labels_

  sample_silhouette_values = silhouette_samples(X, cluster_labels)

  means_lst = []
  for label in range(n_clusters):
      means_lst.append(sample_silhouette_values[cluster_labels == label].mean())

  clusterList = [x for _,x in sorted(zip(means_lst,range(n_clusters)))]

  imageLoona = None
  imageDeukae = None

  images_to_post = pd.read_csv('/content/drive/MyDrive/hourlyloonacatcher/images_to_post.csv').drop(columns='Unnamed: 0').reset_index().drop(columns='index').to_dict(orient='list')
  #df_loona = pd.read_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_loona.csv').drop(columns='Unnamed: 0').reset_index().drop(columns='index')
  #loona, loona_prob = list(df_loona['name']), list(df_loona['prob'])
  #df_deukae = pd.read_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_deukae.csv').drop(columns='Unnamed: 0').reset_index().drop(columns='index')
  #deukae, deukae_prob = list(df_deukae['name']), list(df_deukae['prob'])
  loona, loona_prob, deukae, deukae_prob = get_probs()

  loona_choice = random.choices(loona, weights=loona_prob, k=1) 
  deukae_choice = random.choices(deukae, weights=deukae_prob, k=1) 
  loona_prob, deukae_prob = update_probs(loona_prob, deukae_prob, loona, deukae, loona_choice, deukae_choice)

  for i in range(192):
    print(loona_choice[0], deukae_choice[0]) 
    imageLoona, imageDeukae = -1, -1
    #get image pair for post
    for c in clusterList:
      for image_id, cluster in enumerate(cluster_labels):
          if not dataset["post"][image_id]:
            if cluster == c:
              if loona_choice[0] in dataset["filename"][image_id]:
                print(image_id)
                imageLoona = image_id
                break

      for image_id, cluster in enumerate(cluster_labels):
          if not dataset["post"][image_id]:
            if cluster == c:
              if deukae_choice[0] in dataset["filename"][image_id]:
                print(image_id)
                imageDeukae = image_id
                break

      if (imageLoona != -1) and (imageDeukae != -1):
        dataset["post"][imageLoona] = True
        dataset["post"][imageDeukae] = True
        # new pair
        loona_choice = random.choices(loona, weights=loona_prob, k=1) 
        deukae_choice = random.choices(deukae, weights=deukae_prob, k=1) 
        loona_prob, deukae_prob = update_probs(loona_prob, deukae_prob, loona, deukae, loona_choice, deukae_choice)
        # get out and choose right image
        break
    
    if (imageLoona != -1) and (imageDeukae != -1):
      images_to_post['loona'].append(dataset["filename"][imageLoona])
      images_to_post['deukae'].append(dataset["filename"][imageDeukae])
    else:
      print("No more image combinations for the pair")

    #cv2_imshow(dataset["image"][imageLoona])
    #cv2_imshow(dataset["image"][imageDeukae])

  # generate dfs for saving

  del dataset['image']
  df_dataset = pd.DataFrame(dataset)

  df_images_to_post = pd.DataFrame(images_to_post)

  df_loona = {'name': [], 'prob': []}
  df_loona['name'] = loona
  df_loona['prob'] = loona_prob
  df_loona = pd.DataFrame(df_loona)

  df_deukae = {'name': [], 'prob': []}
  df_deukae['name'] = deukae
  df_deukae['prob'] = deukae_prob
  df_deukae = pd.DataFrame(df_deukae)

  df_dataset.to_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_dataset.csv')
  df_images_to_post.to_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/images_to_post.csv')
  df_loona.to_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_loona.csv')
  df_deukae.to_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_deukae.csv')


In [ ]:
# when no images pairs were generated previously
#first_run()

# when image pairs are already being posted
flying_run()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_dataset.csv').drop(columns='Unnamed: 0').reset_index().drop(columns='index')

In [ ]:
len(df[df['post'] == True])

1152